# Iterables and Iterators
Objects that can be used in `for ... in ...` statements are called *iterable*.
<br>
Now we want to take our class from the [1_classes2](../2_advanced_python/1_classes2.ipynb) and make it iterable.

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
    
    def __repr__(self):
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
    def __add__(self, other):
        if isinstance(other, Triple):
            num1 = self.nums[0] + other.nums[0]
            num2 = self.nums[1] + other.nums[1]
            num3 = self.nums[2] + other.nums[2]
            return Triple(num1, num2, num3)
        elif isinstance(other, int):
            return Triple(self.nums[0]+other, self.nums[1]+other, self.nums[2]+other)
        else:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __bool__(self):
        return any(self.nums)
    
    # add the __iter__ method
    def __iter__(self):
        return iter(self.nums)
 

In [ ]:
my_triple = Triple(1, 2, 3)


for value in my_triple:
    print(value)

In [ ]:
iter?

The `__iter__` - magic-method is what makes an object iterable. Behind the scenes, the `iter`-function calls this method to get the iterator.

An **iterator** is an object that implements `__next__`.  
This is how `__next__` would look like:

In [ ]:
class myrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __next__(self):
        if self.i < self.n:
            self.i += 1
            return self.i
        else:
            raise StopIteration()

In [ ]:
a = myrange(2)

Usually, you want to make an iterator also iterable by returning itself from `__iter__`. Here an example of how to create your own `range`-function:

In [ ]:
class myrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __iter__(self):
        return self

    def __next__(self):
        if self.i < self.n:
            self.i += 1
            return self.i
        else:
            raise StopIteration()

In [ ]:
for i in myrange(5):
    print(i)

Python relies heavily on iterators, and you should use them everytime Python offers them! The following code would be considered *unpythonic*.

In [ ]:
a_list = [10, 20, 30]
for i in range(len(a_list)):
    print(a_list[i])

Instead we prefer using the iterator directly.

In [ ]:
a_list = [10, 20, 30]
for number in a_list:
    print(number)

The iterator keeps its internal state. If we want to start at the beginning again, a fresh iterator will be need. You can try to make the `__iter__` method return a new instance whenever it is called, to get a behaviour like that of built in iterables like lists or ranges.

In [ ]:
a = myrange(5)
next(a)

In [ ]:
for i in a:
    print(i)

In [ ]:
a = myrange(5)
b = range(1,6)

# Both iterables are a representation of the same numbers
print(list(a) == list(b))

# But they still behave differently
for i in b:
    print("range:",i)
    
for i in a:
    print("myrange",i)

## Itertools module
Python provides various [built in iterators](https://docs.python.org/3/library/itertools.html) that we can import and use.<br>

---
# Generators



A Python generator function is a function which returns a generator. Generator functions are implicitly defined by the use of `yield` in the function body. `yield` may be used with a value, in which case that value is treated as the "generated" value. The next time `next()` is called on the generator (i.e. in the next step in a for loop, for example), the generator resumes execution from where it called `yield`, not from the beginning of the function. All of the state, like the values of local variables, is recovered and the generator contiues to execute until the next call to `yield`. 

https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

In [ ]:
def generate_numbers():
    yield 1
    yield 10
    yield 3
    yield 5
    
for i in generate_numbers():
    print(i)

In [ ]:
a = generate_numbers()
print(a)

print(next(a))
print()


In [ ]:
for i in a:
    print(i)
    
print(next(a)) #will throw a StopIteration

When we call a normal Python function, execution starts at the function's first line and continues until a return statement, exception, or when the end of the function is encountered. 
Once a function returns control to its caller, any work done by the function and stored in local variables is lost. A new call to the function creates everything from scratch. 

A **generator** is a certain kind of function (recognized by the keyword *yield* in place of *return*), that doesn't lose its data. If a generator is called, it will run until the next occurence of the `yield` keyword. When called again, it starts right after that, and runs until the next occurence of `yield`.

A generator is an iterator, which means you can loop over it, call next(), and use it the way you'd use any other iterator.

In [ ]:
hasattr(a, '__iter__'), hasattr(a, '__next__')

Generators are a perfect way to get rid of too convolutedly nested for-loops:

In [ ]:
nested_list = [[[1, 2, 3], [4, 5, 6]],[[7, 8, 9], [10, 11, 12]]]

In [ ]:
for i in nested_list:
    for j in i:
        for k in j:
            print(k)

In [ ]:
def nested_list_iterator(thelist):
    for i in thelist:
        for j in i:
            for k in j:
                yield k
                
for i in nested_list_iterator(nested_list):
    print(i)

Also, generators are perfect if you have complex stuff to loop over and/or want to be able to simply replace that thing you're looping over:

<div class="alert alert-block alert-info">
<b>Exercise:</b> 
    <br>
   Use a generator to produce even numbers infinitely. Then print the first ten even numbers.
</div>


<div class="alert alert-block alert-success">
<b>Tip:</b> 
    <br>
   

Use a `while True` loop to produce numbers infinitely. Then wrap the generator function in `enumerate` and `break` after the first ten values.
</div>

In [ ]:
def even_numbers():
    #Your Code Here

for i, num in enumerate(even_numbers()):   
    #Your Code Here
    

So a generator is a function that remembers its state in between calls. It's basically the same as this:

In [ ]:
class EvenNumberGenerator():
    def __init__(self):
        self.index = 0
    
    def __call__(self):
        self.index += 2
        return self.index
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.__call__()
        
numgen = EvenNumberGenerator()

In [ ]:
numgen()

In [ ]:
for i, num in enumerate(numgen):
    print(num)
    if i >= 10:
        break